In [ ]:
import os
import gzip

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import Dataset, DataLoader

from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score

In [ ]:
dataset_path = os.path.join(os.getcwd(), "datasets")

In [ ]:
random_seed = 1
torch.manual_seed(random_seed)

tf_noNorm=torchvision.transforms.Compose([
    torchvision.transforms.ToTensor()
])

train_set = torchvision.datasets.FashionMNIST(dataset_path, train=True, download=True , transform=tf_noNorm)
test_set = torchvision.datasets.FashionMNIST(dataset_path, train=False, download=True , transform=tf_noNorm)

### Categorie
```
0 T-shirt/top
1 Trouser
2 Pullover
3 Dress
4 Coat
5 Sandal
6 Shirt
7 Sneaker
8 Bag
9 Ankle boot
```

In [ ]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [ ]:
batch_size_train = 64
train_loader = DataLoader(train_set, batch_size=batch_size_train)

batch_size_test = 1000
test_loader = DataLoader(test_set, batch_size=batch_size_test, shuffle=True)

## SHOW SOME DATA

In [ ]:
# Show sample
examples = enumerate(train_loader)
batch_idx, (example_data, example_target) = next(examples)

f, axs = plt.subplots(3,3)
plt.tight_layout()
k = 0
for i in range(3):
    for j in range(3):
        axs[i][j].imshow(example_data[k][0], cmap='gray', interpolation='none')
        axs[i][j].set_title("Ground Truth: {}".format(example_target[k]))
        k += 1
plt.show()

## NET ARCH

input
28×28 

output
10 

In [ ]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        
        # conv parameters
        self.ks1 = 5
        self.nk1 = 10
        self.ks2 = 5
        self.nk2 = 20
        self.out_conv1 = (int)((28 - self.ks1 + 1)/2)
        self.out_conv2 = (int)((self.out_conv1 - self.ks2 + 1)/2)
        self.n_unit1 = self.nk2*self.out_conv2**2
        self.n_unit2 = 50
        
        self.conv1 = nn.Conv2d(1, self.nk1, kernel_size=self.ks1)
        self.conv2 = nn.Conv2d(self.nk1, self.nk2, kernel_size=self.ks2)
        self.fc1 = nn.Linear(self.n_unit1, self.n_unit2)
        self.fc2 = nn.Linear(self.n_unit2, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.max_pool2d(x, 2)
        x = F.relu(x)
        x = x.view(-1, self.n_unit1)
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        x = self.fc2(x)
        return F.log_softmax(x)

In [ ]:
def train(*, ld, net, opt, epochs):
    for epoch in range(1, epochs + 1):
        net.train()
        for batch_idx, (data, target) in enumerate(ld):
            opt.zero_grad()
            output = net(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            opt.step()

            if batch_idx % 50 == 0:
                num_campioni_elaborati = batch_idx * len(data)
                num_campioni_totali = len(ld.dataset)
                print('Train Epoch: {} [{}/{}  ({:.0f}%)]  \tLoss: {:.6f}'.format(
                    epoch, num_campioni_elaborati, num_campioni_totali , 
                    100. * num_campioni_elaborati / num_campioni_totali, 
                    loss.item()
                    )
                )
         

In [ ]:
def test(*,tls=None, net, ld):
    net.eval()
    test_loss = 0
    correct = 0
    all_preds = []
    all_targets = []

    with torch.no_grad():
        for data, target in ld:
            output = net(data)
            test_loss += F.nll_loss(output, target, size_average=False).item()
            pred = output.data.max(1, keepdim=True)[1]
            all_preds += pred
            all_targets += target

    test_loss /= len(ld.dataset)
    correct = accuracy_score(all_targets, all_preds, normalize=False)
    accuracy = 100. * correct / len(ld.dataset)
    precision = 100*precision_score(all_targets, all_preds, average='macro')
    recall = 100*recall_score(all_targets, all_preds, average='macro')
    return np.array([test_loss, accuracy, precision, recall, correct])

## 10-FOLD CROSS VALIDATION ##

In [ ]:
def kfold_cv(*, train, valid, model, dataset, bs, epochs, k_fold=5):

    train_score = pd.Series()
    val_score = pd.Series()

    total_size = len(dataset)
    seg = int(total_size/k_fold)
    # tr:train,val:valid; r:right,l:left;  eg: tr_rr: right index of right side train subset
    # index: [tr_ll,tr_lr],[val_l,val_r],[tr_rl,tr_rr]
    for i in range(k_fold):
        tr_ll = 0
        tr_lr = i * seg
        val_l = tr_lr
        val_r = val_l + seg
        tr_rl = val_r
        tr_rr = total_size
        print(f"-------  K-FOLD CV - {i}  -------")

        train_indices = list(range(tr_ll,tr_lr)) + list(range(tr_rl,tr_rr))
        val_indices = list(range(val_l,val_r))

        train_set = torch.utils.data.dataset.Subset(dataset,train_indices)
        val_set = torch.utils.data.dataset.Subset(dataset,val_indices)

        tr_ld = torch.utils.data.DataLoader(train_set, batch_size=bs, shuffle=True, num_workers=4)
        val_ld = torch.utils.data.DataLoader(val_set, batch_size=int(len(val_set)/10), shuffle=True, num_workers=4)

        network = model()
        optimizer = optim.SGD(network.parameters(), lr=0.01, momentum=0.5 )

        print(f"Start Training for indices: [{tr_ll},{tr_lr}),[{tr_rl},{tr_rr})")
        train(net=network, opt=optimizer, ld=tr_ld, epochs=epochs)
        
        print(f"Start Validation for indices: [{val_l},{val_r})")
        val_acc = valid(net=network, ld=val_ld)
        
        print(f"Avg.Loss = {val_acc[0]:.2f}, Accuracy = {val_acc[1]:.2f}%, Precision = {val_acc[2]:.2f}%, Recall = {val_acc[3]:.2f}%, Correct = {int(val_acc[4])}")
        val_score.at[i] = val_acc
        
        print()
    return val_score

In [ ]:
# collect results
kfold = 2
score = kfold_cv(train=train, valid=test, model=Net,dataset=train_set, bs=64, epochs=1, k_fold=kfold)
score = np.array(score)
avg = np.mean(score, axis=0)
print("\nResults:")
print(f"Avg.Loss = {avg[0]:.2f}, Avg.Accuracy = {avg[1]:.2f}%, Avg.Precision = {avg[2]:.2f}%, Avg.Recall = {avg[3]:.2f}%,  Avg.Correct = {avg[4]:.2f}/{int(len(train_set)/kfold)}")


## Train and Validation ##

In [ ]:
network = Net()
optimizer = optim.SGD(network.parameters(), lr=0.01, momentum=0.5 )

train(net=network, opt=optimizer, ld=train_loader, epochs=2)

all_targets = []
all_preds = []
for batch_idx, (x, target) in enumerate(test_loader):
    output = network(x)
    pred = output.data.max(1, keepdim=True)[1]
    all_preds += pred
    all_targets += target


In [ ]:
conf_matrix = confusion_matrix(all_targets, all_preds)
print(f"Confusion Matrix: \n{conf_matrix}")
print(f"Recall = {100*recall_score(all_targets, all_preds, average='macro'):.2f}%")
print(f"Precision = {100*precision_score(all_targets, all_preds, average='macro'):.2f}%")
print(f"Accuracy = {100*accuracy_score(all_targets, all_preds):.2f}%")